In [20]:
from pymongo import MongoClient
from elasticsearch import Elasticsearch
from basic.claims import connect #, get_claims, get_users, get_kibana_gate, products_bankiros, nulls, get_medium, get_claims_raw
# from basic.external import cpa_parse, get_yandex_data
# from basic.dscore import cre_get, mfo_score
from basic.reports import cre_keyword_report, master_report, registrations_medium
from basic.vars import regions_mapper, claim_offers
from bson.objectid import ObjectId
import requests
import json
from datetime import datetime, date, timedelta
import pandas as pd
from time import sleep
from base64 import b64encode
import numpy as np
# from robobrowser import RoboBrowser
import re
# import csv, sqlite3
# import sqlalchemy
import matplotlib.pyplot as plt
import seaborn as sns
from calendar import monthrange
import pickle
import os.path
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from dateutil.relativedelta import relativedelta
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth',1000)
# from IPython.display import Filelink, FileLinks


client, es = connect()

Доспуп к Эластику 	 True
Доспуп к Монге 		 True


In [27]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date",
    date1 ="2021-01-04",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            #'area':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1




INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [28]:
df['visits'].sum()

60630.0

In [29]:
df['date']  =pd.to_datetime(df['date'],format='%Y-%m-%d')

In [30]:
from datetime import datetime
def income_group(date):
    if datetime(2021,3,21) <= date <= datetime(2021,4,19) : return 'Мартовская рассылка'
    if datetime(2021,4,20)<= date <= datetime(2021,5,16): return 'Апрельская рассылка'
    if datetime(2021,5,17) <= date <= datetime(2021,6,20): return 'Майская рассылка'
    if datetime(2021,6,21) <= date<= datetime(2021,7,19): return'Июньская рассылка'
    if datetime(2021,7,20) <= date <= datetime(2021,8,20): return'Июльская рассылка'
    if datetime(2021,8,21) <= date <= datetime(2021,9,25): return'Августcкая рассылка'
    if datetime(2021,9,26) <= date <= datetime(2021,10,18): return'Сентябрьская рассылка'
    if datetime(2021,10,19) <= date <= datetime(2021,11,20): return'Октябрьская рассылка'
    if datetime(2021,11,21) <= date <= datetime(2021,12,20): return'Ноябрьская рассылка'



    else: 
        return 'не определено'
    
df['income_group_first'] = df['date'].apply(income_group)
df

,date,visits,week,income_group_first
0,2021-01-04,2.0,2,не определено
1,2021-01-05,3.0,2,не определено
2,2021-01-06,2.0,2,не определено
3,2021-01-07,3.0,2,не определено
4,2021-01-08,3.0,2,не определено
5,2021-01-10,4.0,2,не определено
6,2021-01-11,2.0,3,не определено
7,2021-01-12,3.0,3,не определено
8,2021-01-13,1.0,3,не определено
9,2021-01-14,2.0,3,не определено


In [31]:
from datetime import datetime
def income_group(firstvisitdate):
    if datetime(2021,3,21) <= firstvisitdate <= datetime(2021,4,19) : return 'Мартовская рассылка'
    if datetime(2021,4,20)<= firstvisitdate <= datetime(2021,5,16): return 'Апрельская рассылка'
    if datetime(2021,5,17) <= firstvisitdate <= datetime(2021,6,20): return 'Майская рассылка'
    if datetime(2021,6,21) <= firstvisitdate<= datetime(2021,7,19): return'Июньская рассылка'
    if datetime(2021,7,20) <= firstvisitdate <= datetime(2021,8,20): return'Июльская рассылка'
    if datetime(2021,8,21) <= firstvisitdate <= datetime(2021,9,25): return'Августcкая рассылка'
    if datetime(2021,9,26) <= firstvisitdate <= datetime(2021,10,18): return'Сентябрьская рассылка'
    if datetime(2021,10,19) <= firstvisitdate <= datetime(2021,11,20): return'Октябрьская рассылка'
    if datetime(2021,11,21) <= firstvisitdate <= datetime(2021,12,20): return'Ноябрьская рассылка'

    



    else: 
        return 'не определено'
    
df['income_group_dt'] = df['date'].apply(income_group)

def income_group_number(income_group_dt):
    if income_group_dt=='Мартовская рассылка' : return 1
    if income_group_dt=='Апрельская рассылка' : return 2
    if income_group_dt=='Майская рассылка' : return 3
    if income_group_dt=='Июньская рассылка' : return 4
    if income_group_dt=='Июльская рассылка' : return 5
    if income_group_dt=='Августcкая рассылка' : return 6
    if income_group_dt=='Сентябрьская рассылка' : return 7
    if income_group_dt=='Октябрьская рассылка' : return 8
    if income_group_dt=='Ноябрьская рассылка' : return 9




    
   


    else: 
        return 'не определено'
    
df['income_group_number'] = df['income_group_dt'].apply(income_group_number)
df
df.to_excel('Tele2 MZ/Теле2_мз_траф.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(0) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_траф.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_траф2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(1) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_траф2.xlsx')
set_with_dataframe(worksheet, df1)


In [32]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:regionArea",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][2]["name"],
            'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','area'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_гор.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(2) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_гор.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_гор.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(3) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_гор.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [33]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:gender",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            'gender':result[i]["dimensions"][1]["name"],
            #'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][3]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','gender'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_пол.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(4) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_пол.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_пол2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(5) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_пол2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [34]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:interest",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'area':result[i]["dimensions"][1]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','interest'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_инт.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(6) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_инт.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_инт2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(7) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_инт2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [35]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:interest2d1,ym:s:interest2d2",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           #'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','interest','interest2'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_инте.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(8) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_инте.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})
df1.to_excel('Tele2 MZ/Теле2_мз_инте2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(9) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_инте2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [36]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:ageInterval",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'age_group':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
#df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','age_group'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_воз.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(10) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_воз.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_воз2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(11) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_воз2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [37]:
from tapi_yandex_metrika import YandexMetrikaStats
import json
import pandas as pd
ACCESS_TOKEN = "AQAAAAABcmq_AAWE17Za_NIhiUS8npNKvsLyVsQ"
METRIC_IDS = "48628676"
# По умолчанию возвращаются только 10000 строк отчета, 
# если не указать другое кол-во в параметре limit.
# В отчете может быть больше строк, чем указано в limit 
# Тогда необходимо сделать несколько запросов для получения всего отчета.
# Чтоб сделать это автоматически вы можете указать 
# параметр receive_all_data=True при инициализации класса.
#Параметры запроса для библиотеки tapi_yandex_metrika
api = YandexMetrikaStats(
    access_token=ACCESS_TOKEN, 
    # Если True, будет скачивать все части отчета. По умолчанию False.
    receive_all_data=True
)
#Параметры запроса для библиотеки tapi_yandex_metrika
params = dict(
    ids = METRIC_IDS,
    metrics = "ym:s:users",
    dimensions = "ym:s:date,ym:s:mobilePhone",
    date1 ="2021-01-01",
    date2 = "today",
    group = "week",
    sort = "ym:s:date",
    accuracy="full",
    limit =100000,
    #filters = "ym:s:lastTrafficSource=='organic'"
    filters = "ym:s:UTMSource=='tele2'"
)
#Получаем данные из Yandex.Metrika API
result = api.stats().get(params=params)
result = result().data
result = result[0]['data']
#Создаем пустой dict (словать данных)
dict_data = {}
#Парсим исходный list формата Json в dictionary (словарь данных)
for i in range(0, len(result)-1):
    dict_data[i] = {
            'date':result[i]["dimensions"][0]["name"],
           'phone':result[i]["dimensions"][1]["name"],
            #'gender':result[i]["dimensions"][1]["name"],
            #'interest2':result[i]["dimensions"][2]["name"],
        #'city':result[i]["dimensions"][5]["name"],
            #'interest':result[i]["dimensions"][1]["name"],
        'visits':result[i]["metrics"][0],
        #'gender':result[i]["dimensions"][4]["name"],
        #'firstvisitdate':result[i]["dimensions"][6]["name"],
        #'visitid':result[i]["dimensions"][7]["name"],
        
          }
#Создаем DataFrame из dict (словаря данных или массива данных)
dict_keys = dict_data[0].keys()
df = pd.DataFrame.from_dict(dict_data, orient='index',columns=dict_keys)
#df = df[df['interest']=='Finance']
df['date'] = pd.to_datetime(df['date'])
df['week'] =(df['date'].dt.week)+1
df['income_group_first'] = df['date'].apply(income_group)
df = df.groupby(['income_group_first','phone'],as_index=False).agg({'visits':'sum'})
df
df.to_excel('Tele2 MZ/Теле2_мз_дев.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(12) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_дев.xlsx')
set_with_dataframe(worksheet, df1)

df1 = df
df1 = df1.rename(columns={
                        "income_group_first": "income_group_first1"})

df1.to_excel('Tele2 MZ/Теле2_мз_дев2.xlsx')
import gspread
from gspread_dataframe import set_with_dataframe

# Доступ к GS
gc = gspread.service_account('voltaic-syntax-302214-e18198d5466a.json')
sh = gc.open_by_key('1Ud1NDSQ7qMlZqVKPKi4nulx2z9Ay0b0BNNTPxzK1F28')
worksheet = sh.get_worksheet(13) #-> 0 - лист один, 1 - второй и тд.

# Выбор датафрейма
df1 = pd.read_excel('Tele2 MZ/Теле2_мз_дев2.xlsx')
set_with_dataframe(worksheet, df1)

INFO:root:Наличие семплирования: False
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.


In [38]:
df

,income_group_first,phone,visits
0,Августcкая рассылка,ASUS,28.0
1,Августcкая рассылка,Alcatel,26.0
2,Августcкая рассылка,Apple,868.0
3,Августcкая рассылка,BQ,81.0
4,Августcкая рассылка,Blackview,12.0
5,Августcкая рассылка,DEXP,4.0
6,Августcкая рассылка,Digma,2.0
7,Августcкая рассылка,Doogee,7.0
8,Августcкая рассылка,Explay,3.0
9,Августcкая рассылка,Fly,9.0
